# 用Tensorflow实现seq2seq

```
这篇博客记录了我在尝试用tensorflow实现seq2seq时的学习心得。
```

## 1.引入

最近想用Tensorflow实现一个Seq2Seq模型，之前用Tensorflow编写过简单的RNN，本以为应该差不多，但实际研究之后，发现有很多“坑”和细节需要注意，于是开这篇博客记录学习过程。

## 2.seq2seq简介

seq2seq从字面上理解就是序列到序列，也就是说，输入是一串序列，输出也是一串序列。但如果只是这样的话，RNN不是就够了吗？别着急，这里还涉及到另外两个概念：**encoder和decoder**。所谓encoder是可以理解成为特征提取器，把序列的特征提取到最后一个cell的hidden state里，decoder可以理解成为sequence 生成器。参考paper[Sequence to Sequence Learning with Neural Networks](http://papers.nips.cc/paper/5346-information-based-learning-by-agents-in-unbounded-state-spaces.pdf)

请看这张图：
![图一](https://raw.githubusercontent.com/applenob/nlp_projects/master/resources/basic_seq2seq.png)

这就是最基础的seq2seq。但如果只看这张图，实际上不还是RNN吗？只不过要指定后面哪几个cell要输出。

再看另外一张图：

![图二](https://raw.githubusercontent.com/applenob/nlp_projects/master/resources/2-seq2seq-feed-previous.png)

注意decoder的部分，当前cell的输出，转换成预测结果以后，再作为输入送给下一个cell；而普通的RNN是直接由当前cell的输出作为输入送给下一个cell，所以这里有一个关键不同。

那么这样的decoder如何实现呢？在训练的时候很容易解决，因为训练时我们知道输出的句子是什么，所以可以直接作为decoder的input送给decoder；但是在预测的时候呢？

来看一张来自最新的seq2seq的paper的图片，这篇[Neural Machine Translation and Sequence-to-sequence Models: A Tutorial](https://arxiv.org/pdf/1703.01619.pdf)也是强推。

## 实现attention

attention的原理有一篇比较不错的[科普文](http://blog.csdn.net/malefactor/article/details/50550211)。

具体实现参考Google的[Grammar as a Foreign Language](https://arxiv.org/pdf/1412.7449.pdf)的2.1节。

这里做简短摘要：

- input words: $1, . . . , T_A$

- encoder的hidden state：$h_1, . . . , h_{T_A}$ 

- decoder的hidden state：$ d_1, . . . , d_{T_B} := h_{T_A+1} , . . . , h_{T_A+T_B} $

定义，对每个时间t，有：

$u_i^t = v^Ttanh(W_1'h_i+W_2'd_t)$

$a_i^t = softmax(u_i^t)$

$d_t' = \sum{a_i^th_i}_{i=1}^{T_A}$

其中$v$,$W_1'$,$W_2'$是要学习的参数，向量$u^t$的长度是$T_A$，第i个元素代表要分配给encoder hidden state $h_i$；

最后将[$d_t$和$d_t'$]concatenate在一起，作为新的decoder的hidden state。

In [1]:
import tensorflow as tf

In [ ]:
tf.nn.